# Лабораторная работа №7 PyTorch
Группа 45/2
---

**Импорт библиотек**

In [79]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**Задание 1.**
Загрузить набор данных MNIST, который включает в себя рукописные цифры от 0 до 9.

In [80]:
# Определяю преобразования данных, которые будут применены к изображениям MNIST
transform = transforms.Compose([
    transforms.ToTensor(),  # Преобразование изображения в тензор
    transforms.Normalize((0.5,), (0.5,))  # Нормализация значений пикселей к диапазону [-1, 1]
])

# Загрузка набор данных MNIST
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Создаю DataLoader для обучающего и тестового наборов данных
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

**Задание 2.**
Предобработать данные, чтобы они были приведены к нужному формату и масштабу.

In [81]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    #transforms.Flatten()  # Разглаживание изображений
    transforms.Resize((500, 500))  # Изменение размера изображений
])

**Задание 3.**
Создать модель нейронной сети с использованием PyTorch. Модель должна содержать несколько слоев, включая скрытые слои.

In [82]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)  # Полносвязный слой с 784 входами и 256 выходами
        self.fc2 = nn.Linear(256, 128)  # Полносвязный слой с 256 входами и 128 выходами
        self.fc3 = nn.Linear(128, 10)  # Полносвязный слой с 128 входами и 10 выходами (для 10 классов)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Разглаживание входных данных (приведение в одномерный вектор)
        x = F.relu(self.fc1(x))  # Применение функции активации ReLU к первому слою
        x = F.relu(self.fc2(x))  # Применение функции активации ReLU ко второму слою
        x = self.fc3(x)  # Выходной слой без функции активации (например, для использования CrossEntropyLoss)
        return x

- nn.Linear(in_features, out_features) - полносвязный слой, где in_features - количество входов, out_features - количество выходов.
- F.relu() - функция активации ReLU.
- view(-1, 28 * 28) - разглаживание входных данных в одномерный вектор.

**Задание 4.**
Обучить модель на тренировочном наборе данных, используя функцию потерь и оптимизатор из PyTorch.

In [83]:
# Создание экземпляра модели
model = NeuralNetwork()

# Определение функции потерь
criterion = nn.CrossEntropyLoss()

# Определение оптимизатора (например, стохастический градиентный спуск)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Цикл обучения
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(images)
        
        # Вычисление функции потерь
        loss = criterion(outputs, labels)
        
        # Обратное распространение
        loss.backward()
        
        # Обновление весов
        optimizer.step()

    # Вывод информации о процессе обучения (например, потери на эпохе)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/10], Loss: 0.4748479425907135
Epoch [2/10], Loss: 0.2909955084323883
Epoch [3/10], Loss: 0.32635822892189026
Epoch [4/10], Loss: 0.186576709151268
Epoch [5/10], Loss: 0.054498091340065
Epoch [6/10], Loss: 0.11743377149105072
Epoch [7/10], Loss: 0.43036291003227234
Epoch [8/10], Loss: 0.2416214644908905
Epoch [9/10], Loss: 0.18230295181274414
Epoch [10/10], Loss: 0.026439432054758072


**Задание 5.**
Оценить качество модели на тестовом наборе данных.

In [84]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Accuracy on the test set: 94.94%


Код считает точность модели на тестовом наборе данных. Он использует torch.no_grad(), чтобы предотвратить вычисление градиентов при прямом проходе через модель, поскольку в данном контексте нам не нужно обновлять веса.

**Задание 6.**
Изменить параметры модели (например, число скрытых слоев, количество нейронов в слоях) и сравнить их влияние на качество распознавания.

In [85]:
# Определение первой модели с ReLU
class ModelReLU(nn.Module):
    def __init__(self):
        super(ModelReLU, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Определение второй модели с сигмоидальной функцией активации
class ModelSigmoid(nn.Module):
    def __init__(self):
        super(ModelSigmoid, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

# Функция обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Функция оценки модели
def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

**Задание 7.**
Изменить функцию активации в нейронной сети (например, ReLU, сигмоида) и сравнить их влияние на качество распознавания.

In [86]:
# Определение критерия и оптимизатора
criterion = nn.CrossEntropyLoss()

# Обучение и оценка первой модели с ReLU
model_relu = ModelReLU()
optimizer_relu = optim.SGD(model_relu.parameters(), lr=0.01)
train_model(model_relu, train_loader, criterion, optimizer_relu)
accuracy_relu = evaluate_model(model_relu, test_loader)

# Обучение и оценка второй модели с сигмоидальной функцией активации
model_sigmoid = ModelSigmoid()
optimizer_sigmoid = optim.SGD(model_sigmoid.parameters(), lr=0.01)
train_model(model_sigmoid, train_loader, criterion, optimizer_sigmoid)
accuracy_sigmoid = evaluate_model(model_sigmoid, test_loader)

# Сравнение результатов
print(f'Accuracy with ReLU: {accuracy_relu * 100:.2f}%')
print(f'Accuracy with Sigmoid: {accuracy_sigmoid * 100:.2f}%')


Accuracy with ReLU: 92.66%
Accuracy with Sigmoid: 68.06%


Результаты отражают различия в производительности моделей с разными функциями активации. В данном случае, высокая точность с ReLU по сравнению с сигмоидальной функцией активации может быть объяснена следующими аспектами:

1. **Преимущества ReLU:**
   - ReLU (Rectified Linear Unit) обычно показывает хорошую производительность в нейронных сетях и может способствовать быстрой сходимости модели.
   - ReLU не сталкивается с проблемой затухания градиентов, которая может возникнуть при использовании сигмоиды, особенно в глубоких сетях.

2. **Сигмоидальная функция активации:**
   - Сигмоидальная функция ограничивает значения в интервале (0, 1). Это может привести к проблеме затухания градиентов, особенно при обратном распространении ошибки через множество слоев.
   - В данном случае, сигмоидальная функция активации, возможно, слишком сильно ограничивает выходные значения, затрудняя обучение модели.